### Zigbang 원룸 매물 데이터 수집

In [2]:
import requests 
import pandas as pd 

#### Process
    - 동이름으로 위도 경도 구하기
    - 위도 경도로 geohash 알아내기
    - geohash로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기

#### 1. 동이름으로 위도 경도 구하기

In [5]:

addr = '망원동'
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
response = requests.get(url)
data = response.json()['items'][0]
lat, lng = data['lat'], data['lng']
lat, lng

(37.556785583496094, 126.9013442993164)

#### 2. 위도 경도로 geohash 알아내기

In [7]:
!pip install geohash2

In [8]:

import geohash2
geohash = geohash2.encode(lat,lng,precision = 5)
geohash

'wydjx'

#### 3. geohash로 매물 아이디 가져오기

In [10]:
# url이 인코딩되있는 경우 디코딩 과정 필요 
url = 'https://apis.zigbang.com/v2/items/oneroom?geohash=wydm&depositMin=0&rentMin=0&salesTypes[0]=전세&salesTypes[1]=월세&lngEast=126.92783575805556&lngWest=126.78338880611302&latSouth=37.47632007204551&latNorth=37.56054215395402&domain=zigbang&checkAnyItemWithoutFilter=true'

response = requests.get(url)
response

<Response [200]>

In [12]:
ids = [item['itemId'] for item in response.json()['items']]
len(ids) , ids[:5]
ids=ids[:300]

#### 4. 매물 아이디로 매물 정보 가져오기

In [15]:
url = 'https://apis.zigbang.com/v2/items/list'
#params = {'domain':'zigbang', 'item_ids':ids}
params = {'domain':'zigbang', 'item_ids':ids[:5]} # 최그 가져올 수 있는 dic len 이 15개로 한정되었기 때문에 5개만 가져오기 
response = requests.post(url,params)
response 

<Response [200]>

In [17]:
response.text #

'{"items":[{"item_id":42123052,"section_type":null,"images_thumbnail":"https://ic.zigbang.com/ic/items/42123052/1.jpg","sales_type":"월세","sales_title":"월세","deposit":500,"rent":45,"size_m2":23.14,"공급면적":{"m2":23.14,"p":"7"},"전용면적":{"m2":23.14,"p":"7"},"계약면적":null,"room_type_title":null,"floor":"2","floor_string":"2","building_floor":"4","title":"신흥역 2분 , 넓은 옵션 원룸,즉시입주","is_first_movein":null,"room_type":"01","status":true,"tags":[],"service_type":"원룸","random_location":{"lat":37.4414801397193,"lng":127.14587132583516},"location":{"lat":37.4414801397193,"lng":127.14587132583516},"manage_cost":"0","reg_date":"2024-09-06T11:03:00+09:00","is_new":false,"addressOrigin":{"local1":"경기도","local2":"성남시 수정구","local3":"신흥동","local4":"","fullText":"경기도 성남시 수정구 신흥동","localText":"성남시 수정구 신흥동"},"action":{"isRead":false,"readAt":null,"isInquired":false,"inquiredAt":null,"isRewarded":false,"rewardedAt":null,"isReported":false,"reportedAt":null,"isChecked":false,"checkedAt":null,"isZzim":false},"contrac

In [19]:
columns = ['item_id', 'deposit', 'rent', 'size_m2', 'address1']
data = response.json()['items']
df = pd.DataFrame(data)[columns]
df.head()

,item_id,deposit,rent,size_m2,address1
0,42123052,500,45,23.14,경기도 성남시 수정구 신흥동
1,42022206,39000,0,140.00,경기도 성남시 수정구 신흥동
2,42200778,39000,0,105.00,경기도 성남시 수정구 신흥동
3,42237485,39000,0,157.28,경기도 성남시 수정구 신흥동
4,41955717,5000,55,29.75,경기도 성남시 수정구 신흥동


In [21]:
len(ids)

300

In [23]:
page_block = 15
print(list(range(0,len(ids), 15)))

[0, 15, 30, 45, 60, 75, 90, 105, 120, 135, 150, 165, 180, 195, 210, 225, 240, 255, 270, 285]


In [25]:
datalist = []
for start in range(0,len(ids), 15):
    end = start + page_block
    # print(start,end)
    url = 'https://apis.zigbang.com/v2/items/list'
    params = {'domain':'zigbang', 'item_ids':ids[start:end]} # 데이터프레임1개 당 15개씩 
    response = requests.post(url,params)
    columns = ['item_id', 'deposit', 'rent', 'size_m2', 'address1']
    data = response.json()['items']
    df = pd.DataFrame(data)[columns]#
    datalist.append(df)

In [30]:
result = pd.concat(datalist, ignore_index = True)
result

,item_id,deposit,rent,size_m2,address1
0,42123052,500,45,23.14,경기도 성남시 수정구 신흥동
1,42022206,39000,0,140.00,경기도 성남시 수정구 신흥동
2,42200778,39000,0,105.00,경기도 성남시 수정구 신흥동
3,42237485,39000,0,157.28,경기도 성남시 수정구 신흥동
4,41955717,5000,55,29.75,경기도 성남시 수정구 신흥동
...,...,...,...,...,...
295,42375512,200,30,16.50,서울시 관악구 신림동
296,41813997,4000,10,19.00,서울시 관악구 신림동
297,42037397,3000,10,19.00,서울시 관악구 신림동
298,42254912,100,40,17.00,서울시 관악구 신림동
